In [1]:
import pandas as pd
import numpy as np

In [2]:
archivo_entrada = "/home/fernando/git/ag/zona_A/entrada"
#carga como df del archivo de entrada, el cual debe tener el formato:
entrada_df= pd.read_table(archivo_entrada)

#agrego cuatro columnas con las posiciones iniciales y finales para los hits de proteina y rna.
#primero busca con un rfind, y luego a ese numero le suma el largo del hit.
entrada_df["prot_hit_s"] = entrada_df[["prot_seq", "prot_hit"]].apply(lambda x: x[0].find(x[1]), axis=1)
entrada_df["prot_hit_e"] = entrada_df[["prot_hit", "prot_hit_s"]].apply(lambda x: len(x[0]) + x[1], axis=1)
entrada_df["rna_hit_s"] = entrada_df[["rna_seq", "rna_hit"]].apply(lambda x: x[0].find(x[1]), axis=1)
entrada_df["rna_hit_e"] = entrada_df[["rna_hit", "rna_hit_s"]].apply(lambda x: len(x[0]) + x[1], axis=1)


#subset y separacion de tabla inicial en proteina y rna
prot_df = entrada_df[["sp", "prot_seq", "prot_hit_s", "prot_hit_e"]]
rna_df = entrada_df[["sp", "rna_seq", "rna_hit_s", "rna_hit_e"]]


Reformateo los data frame para obtener un df (para cada uno) con el formato necesario para ggplot,
esto es una residuo o nucleotido por fila, y una columna color donde 1 es que esa posicion es un hit.

In [3]:
# Proteina
prot_df_salida = pd.DataFrame()
for i in range(len(prot_df)):
    virus = prot_df.iloc[i, 0]
    n_virus = i + 1
    seq_list = list(prot_df.iloc[i, 1])
    l = len(prot_df.iloc[i, 1])
    hit_s = prot_df.iloc[i, 2]
    hit_e = prot_df.iloc[i, 3]
    x = range(1,l + 1)
    tmp = pd.DataFrame({"x": x, "sp_id": [n_virus] * l, "sp": [virus] * l, "seq": seq_list, "color": [0] * l})
    tmp["color"][hit_s:hit_e] = [1] * (hit_e - hit_s)
    prot_df_salida = pd.concat([prot_df_salida, tmp])

/home/fernando/.local/lib/python2.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [4]:
# RNA
rna_df_salida = pd.DataFrame()
for i in range(len(rna_df)):
    virus = rna_df.iloc[i, 0]
    n_virus = i + 1
    seq_list = list(rna_df.iloc[i, 1])
    l = len(rna_df.iloc[i, 1])
    l_aa = l / 3
    x_rna = np.array([])
    hit_s = rna_df.iloc[i, 2]
    hit_e = rna_df.iloc[i, 3]
    for e in range(l_aa + 1):
        x_rna = np.append(x_rna, np.array([1.25, 1.5, 1.75]) + e)
    x_rna = x_rna[:l]
    tmp = pd.DataFrame({"x": x_rna, "sp_id": [n_virus] * l, "sp": [virus] * l, "seq": seq_list, "color": [0] * l})
    tmp["color"][hit_s:hit_e] = [2] * (hit_e - hit_s)
    rna_df_salida = pd.concat([rna_df_salida, tmp])

/home/fernando/.local/lib/python2.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [5]:
#reordenar las columnas
prot_df_salida = prot_df_salida[["sp", "sp_id", "x", "seq", "color"]]
rna_df_salida = rna_df_salida[["sp", "sp_id", "x", "seq", "color"]]

prot_df_salida.to_csv("//home/fernando/git/ag/zona_A/aa.csv", index=False)
rna_df_salida.to_csv("/home/fernando/git/ag/zona_A/rna.csv", index=False)
